In [ ]:
!uv pip install sentence_transformers openai crewai

In [ ]:
# Set up enviroment variables
import os

os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
os.environ["SERPER_API_KEY"] = "YOUR_SERPER_API_KEY"

In [ ]:
# Our custom vector database

import numpy as np

class VectorDatabase:
    def __init__(self):
        # Store all vectors in an array
        self.vectors = []

    # Add vector to database
    def add_vector(self, vec_id, vector, metadata=None):
        record = {
            "id": vec_id,
            "vector": np.array(vector, dtype=np.float32),
            "metadata": metadata
        }

        self.vectors.append(record)

    # Retreive all vectors from database
    def get_all_vectors(self):
        return self.vectors

    # Calculate consine similarity between vectors
    def _cosine_similarity(self, vec_a, vec_b):
        # Calculate dot product
        dot_product = np.dot(vec_a, vec_b)

        # Calculate the magnitude of vector A
        norm_a = np.linalg.norm(vec_a)

        # Calculate the magnitude of vector B
        norm_b = np.linalg.norm(vec_b)

        cos_sim = dot_product / (norm_a * norm_b + 1e-8)  # small epsilon to avoid division by zero

        return cos_sim

    # Search for similar vectors and return the top_k results
    def search(self, query_vector, top_k = 3):
        query_vector = np.array(query_vector, dtype = np.float32)

        # Stores the top_k results
        results = []

        for record in self.vectors:
            sim = self._cosine_similarity(query_vector, record["vector"])

            results.append({
                "id": record["id"],
                "similarity": sim,
                "metadata": record["metadata"]
            })

        results.sort(key=lambda x: x["similarity"], reverse=True)

        return results[:top_k]

In [ ]:
# Set up our custom vector database
company_db = VectorDatabase()

# Hypothetical company information
company_information = [
    "Quantum Horizons Inc. is a pioneering space exploration company founded in 2030.",
    "The company specializes in developing quantum-powered spacecraft for interplanetary travel.",
    "With a team of 500 aerospace engineers and quantum physicists, Quantum Horizons is pushing the boundaries of space technology.",
    "Their flagship project, the 'StarLeap', aims to reduce travel time to Mars from months to just weeks.",
    "Quantum Horizons has established the first permanent research base on the Moon's far side.",
    "The company's innovative quantum propulsion system has revolutionized the concept of space travel.",
    "Headquartered in a state-of-the-art facility in Houston, Quantum Horizons also maintains orbital research stations.",
    "They've partnered with major space agencies worldwide to advance human presence in the solar system.",
    "Quantum Horizons' CEO, Dr. Zara Novak, is a former astronaut and a leading expert in quantum mechanics.",
    "The company's mission is to make interplanetary travel accessible and establish humanity as a multi-planet species."
]

In [ ]:
# Populate the custom vector database
from sentence_transformers import SentenceTransformer

# Embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

for idx, sentence in enumerate(company_information):
    # Create sentence embedding
    embedding = model.encode(sentence)

    # Add sentence embedding to the database
    company_db.add_vector(vec_id=f"sentence_{idx}", vector=embedding, metadata={"sentence": sentence})

In [ ]:
# Check record details
for record in company_db.get_all_vectors():
    print(record)
    break

{'id': 'sentence_0', 'vector': array([-2.05399971e-02, -3.94914933e-02, -3.26552913e-02, -7.13318447e-03,
       -5.88249825e-02, -6.15303256e-02, -6.49780184e-02, -6.41458109e-02,
        1.72448810e-02, -8.53577163e-03,  3.96655407e-03,  6.54808851e-03,
       -5.97061180e-02,  3.57571431e-02, -7.93242827e-02,  2.50909179e-02,
       -3.04083582e-02, -7.77376667e-02,  5.98660298e-02, -9.73092616e-02,
       -1.31193534e-01,  2.77496316e-02,  3.16398218e-02,  7.13907108e-02,
        5.89662828e-02,  3.68272215e-02,  6.17195060e-03, -3.38188782e-02,
        5.62651493e-02,  6.58059791e-02,  1.55798178e-02,  3.28813791e-02,
        8.33073631e-02, -2.06086300e-02,  1.71197336e-02,  9.28582028e-02,
        2.82636993e-02, -2.45555472e-02, -7.57346898e-02, -6.25496656e-02,
       -5.02399839e-02, -5.66332303e-02,  1.86945889e-02,  7.54210306e-03,
        2.61634719e-02,  5.13240658e-02, -8.16290546e-03,  2.40288470e-02,
        3.12294569e-02, -1.57156903e-02, -5.70101757e-03, -3.85988913

In [ ]:
# Check embedding size of a record
for record in company_db.get_all_vectors():
    print(record["vector"].shape)
    break

(384,)


In [ ]:
# Check number of records in the database
len(company_db.get_all_vectors())

10

# Tools

In [ ]:
# Create custom RAG tool
from crewai.tools import tool
from openai import OpenAI

@tool("RAG Tool")
def rag_tool(question: str) -> str:
    """Tool to search for relevant information from a vector database."""

    # Encode the question
    query_vec = model.encode(question)

    # Get top 5 similar vector
    results = company_db.search(query_vec, top_k = 5)

    # Build context from the results
    context = "\n".join([f"- {res['metadata']['sentence']}" for res in results])

    # Create the prompt
    prompt = f"""You are a helpful assistant. Use the context below to answer the user's question.

            Context:
            {context}

            Question: {question}

            Answer:
            """

    # Generate an answer using the context
    client = OpenAI()

    response = client.responses.create(
        model = "gpt-4o-mini",
        input = prompt
    )

    answer = response.output_text

    # Return the answer
    return answer

In [ ]:
# Test RAG tool

rag_tool.run("What is Quantum Horizons?")

Using Tool: RAG Tool


"Quantum Horizons is a pioneering space exploration company founded in 2030, specializing in advanced space technology. With a team of 500 aerospace engineers and quantum physicists, the company aims to push the boundaries of space exploration. It is led by CEO Dr. Zara Novak, a former astronaut and expert in quantum mechanics. Quantum Horizons has established the first permanent research base on the Moon's far side and operates from a state-of-the-art facility in Houston, as well as maintaining orbital research stations."

In [ ]:
# Create custom Web search tool
import requests

@tool("Web Search Tool")
def web_search_tool(query: str) -> str:
    """Tool to search the web for relevant information."""

    url = "https://google.serper.dev/search"

    headers = {
        "X-API-KEY": os.environ.get("SERPER_API_KEY"),
        "Content-Type": "application/json",
    }

    payload = {
        "q": query
    }

    response = requests.post(url, headers = headers, json = payload)

    if response.status_code != 200:
        raise Exception(f"Request failed with status code: {response.status_code}")

    data = response.json()

    # Get value associated with key "organic" else return []
    search_results = data.get("organic", [])

    if not search_results:
        return "No search results found."

    context = ""

    for result in search_results:
        title = result.get("title", "")
        link = result.get("link", "")
        snippet = result.get("snippet", "")
        context += f"Title: {title}\nLink: {link}\nSnippet: {snippet}\n\n"

    return f"Web Search Results:\n{context}"

In [ ]:
# Test custom Web search tool

web_search_tool.run("Important AI innovations of 2025")

Using Tool: Web Search Tool


"Web Search Results:\nTitle: What's next for AI in 2025 - MIT Technology Review\nLink: https://www.technologyreview.com/2025/01/08/1109188/whats-next-for-ai-in-2025/\nSnippet: You already know that agents and small language models are the next big things. Here are five other hot trends you should watch out for this year.\n\nTitle: 6 AI trends you'll see more of in 2025 - Microsoft News\nLink: https://news.microsoft.com/source/features/ai/6-ai-trends-youll-see-more-of-in-2025/\nSnippet: In 2025, AI will evolve from a tool for work and home to an integral part of both. AI-powered agents will do more with greater autonomy and help simplify your ...\n\nTitle: Five Trends in AI and Data Science for 2025\nLink: https://sloanreview.mit.edu/article/five-trends-in-ai-and-data-science-for-2025/\nSnippet: From agentic AI to unstructured data, these 2025 AI trends deserve close attention from leaders. Get fresh data and advice from two experts.\n\nTitle: Top 7 Forecasted AI Trends To Watch In 2025

In [ ]:
# Create Agents

from crewai import Agent

retriever_agent = Agent(
    role="Retriever Agent",
    goal="Retrieve the most relevant information to answer the user's query: {user_query}",
    backstory=(
        "You're a helpful agent. "
        "You're an expert at finding the right information to answer a user's query. "
        "You are great at following instructions and sequentially picking tools for information retrieval. "
        "You have decades of experience doing this."
    ),
    tools=[rag_tool, web_search_tool],
    verbose=True,
)

customer_support_agent = Agent(
    role="Senior Customer Support Agent",
    goal=(
        "Accurately and concisely answer the user's query: {user_query} using the retrieved information. "
        "If you are unable to answer the query, apologise and tell that you do not have all the information you need to answer the query."
    ),
    backstory=(
        "You are a helpful senior customer support agent. "
        "You have decades of experience in answering user queries grounded to accurate information."
    ),
    verbose=True,
)

In [ ]:
# Create Tasks

from crewai import Task

# Task 1: Retrieval Task
retrieval_task = Task(
    description=(
        "Retrieve the most relevant information from the given sources to answer the user's query: {user_query}. "
        "ALWAYS use the RAG Tool first. "
        "If you cannot find the required information, ONLY THEN use the Web Search Tool. "
        "DO NOT USE the Web Search Tool if you have sufficient information to accurately answer the user's query."
    ),
    expected_output="The most relevant information from the given sources to answer the user's query in a text format.",
    agent=retriever_agent,
)

# Task 2: Customer Support Task
customer_support_task = Task(
    description=(
        "Using the retrieved information, accurately and concisely answer the user's query: {user_query}."
    ),
    expected_output=(
        "Concise and accurate response based on the retrieved information given the user query: {user_query}. "
        "If you are unable to answer the query, apologise and inform the user that you do not have all the necessary information."
    ),
    agent=customer_support_agent,
    context=[retrieval_task],  # This task will use the output from the previous task as its context
)

In [ ]:
# Create Crew

from crewai import Crew
from crewai.process import Process

customer_support_crew = Crew(
    agents = [retriever_agent, customer_support_agent],
    tasks = [retrieval_task, customer_support_task],
    verbose = True,
    process = Process.sequential
)

### Result 1: Asking agent for company related information

In [ ]:
# Crew inputs

crew_inputs = {
    "user_query": "What is the name of the flagship project of the company?",
}

In [ ]:
# Run the crew

result = customer_support_crew.kickoff(inputs = crew_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a1c9c319-ea8c-46c4-9c64-077ed0a7cbb7                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 6a16152d-0ce4-4b1e-83d4-5f39fa526b77
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 6a16152d-0ce4-4b1e-83d4-5f39fa526b77
       Status: Executing Task...
    └── 🤖 Agent: Retriever Agent
            Status: In Progress

# Agent: Retriever Agent
## Task: Retrieve the most relevant information from the given sources to answer the user's query: What is the name of the flagship project of the company?. ALWAYS use the RAG Tool first. If you cannot find the required information, ONLY THEN use the Web Search Tool. DO NOT USE the Web Search Tool if you have sufficient information to accurately answer the user's query.


🚀 Crew: crew
└── 📋 Task: 6a16152d-0ce4-4b1e-83d4-5f39fa526b77
       Status: Executing Task...
    └── 🤖 Agent: Retriever Agent
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Senior Customer Support Agent
## Final Answer: 
The name of the flagship project of Quantum Horizons Inc. is the 'StarLeap'.




🚀 Crew: crew
├── 📋 Task: 6a16152d-0ce4-4b1e-83d4-5f39fa526b77
│      Assigned to: Retriever Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Retriever Agent
│           Status: ✅ Completed
│       └── 🔧 Used RAG Tool (8)
└── 📋 Task: cde4d9db-8cf0-49ec-a96c-a48cff42649f
       Status: Executing Task...
    └── 🤖 Agent: Senior Customer Support Agent
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 6a16152d-0ce4-4b1e-83d4-5f39fa526b77
│      Assigned to: Retriever Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Retriever Agent
│           Status: ✅ Completed
│       └── 🔧 Used RAG Tool (8)
└── 📋 Task: cde4d9db-8cf0-49ec-a96c-a48cff42649f
       Assigned to: Senior Customer Support Agent
       Status: ✅ Completed
    └── 🤖 Agent: Senior Customer Support Agent
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: cde4d9db-8cf0-49ec-a96c-a48cff42649f                                                                     │
│  Agent: Senior Customer Support Agent                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: a1c9c319-ea8c-46c4-9c64-077ed0a7cbb7                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
print(result.raw)

The name of the flagship project of Quantum Horizons Inc. is the 'StarLeap'.


### Result 2: Asking agent for information not related to the company

In [ ]:
# Crew inputs
crew_inputs_2 = {
    "user_query": "Who won the Nobel prize in 2024 in Physics?",
}

# Run the crew
result_2 = customer_support_crew.kickoff(inputs = crew_inputs_2)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 27c31765-1616-457b-82e7-a05a785dd68e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 6a16152d-0ce4-4b1e-83d4-5f39fa526b77
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 6a16152d-0ce4-4b1e-83d4-5f39fa526b77
       Status: Executing Task...
    └── 🤖 Agent: Retriever Agent
            Status: In Progress

# Agent: Retriever Agent
## Task: Retrieve the most relevant information from the given sources to answer the user's query: Who won the Nobel prize in 2024 in Physics?. ALWAYS use the RAG Tool first. If you cannot find the required information, ONLY THEN use the Web Search Tool. DO NOT USE the Web Search Tool if you have sufficient information to accurately answer the user's query.


🚀 Crew: crew
└── 📋 Task: 6a16152d-0ce4-4b1e-83d4-5f39fa526b77
       Status: Executing Task...
    └── 🤖 Agent: Retriever Agent
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Senior Customer Support Agent
## Final Answer: 
The Nobel Prize in Physics for 2024 was awarded jointly to John J. Hopfield and Geoffrey Hinton "for foundational discoveries and inventions that enable machine learning with artificial neural networks."




🚀 Crew: crew
├── 📋 Task: 6a16152d-0ce4-4b1e-83d4-5f39fa526b77
│      Assigned to: Retriever Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Retriever Agent
│           Status: ✅ Completed
│       ├── 🔧 Used RAG Tool (5)
│       └── 🔧 Used Web Search Tool (3)
└── 📋 Task: cde4d9db-8cf0-49ec-a96c-a48cff42649f
       Status: Executing Task...
    └── 🤖 Agent: Senior Customer Support Agent
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 6a16152d-0ce4-4b1e-83d4-5f39fa526b77
│      Assigned to: Retriever Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Retriever Agent
│           Status: ✅ Completed
│       ├── 🔧 Used RAG Tool (5)
│       └── 🔧 Used Web Search Tool (3)
└── 📋 Task: cde4d9db-8cf0-49ec-a96c-a48cff42649f
       Assigned to: Senior Customer Support Agent
       Status: ✅ Completed
    └── 🤖 Agent: Senior Customer Support Agent
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: cde4d9db-8cf0-49ec-a96c-a48cff42649f                                                                     │
│  Agent: Senior Customer Support Agent                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 27c31765-1616-457b-82e7-a05a785dd68e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
print(result_2.raw)

The Nobel Prize in Physics for 2024 was awarded jointly to John J. Hopfield and Geoffrey Hinton "for foundational discoveries and inventions that enable machine learning with artificial neural networks."


### Result 3: Incomprehensible query

In [ ]:
crew_inputs_3 = {
    "user_query": "What is the name of XNNASDMN?",
}

In [ ]:
result_3 = customer_support_crew.kickoff(inputs = crew_inputs_3)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 27c31765-1616-457b-82e7-a05a785dd68e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 6a16152d-0ce4-4b1e-83d4-5f39fa526b77
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 6a16152d-0ce4-4b1e-83d4-5f39fa526b77
       Status: Executing Task...
    └── 🤖 Agent: Retriever Agent
            Status: In Progress

# Agent: Retriever Agent
## Task: Retrieve the most relevant information from the given sources to answer the user's query: What is the name of XNNASDMN?. ALWAYS use the RAG Tool first. If you cannot find the required information, ONLY THEN use the Web Search Tool. DO NOT USE the Web Search Tool if you have sufficient information to accurately answer the user's query.


🚀 Crew: crew
└── 📋 Task: 6a16152d-0ce4-4b1e-83d4-5f39fa526b77
       Status: Executing Task...
    └── 🤖 Agent: Retriever Agent
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Senior Customer Support Agent
## Final Answer: 
I apologize, but I do not have all the information I need to answer the query regarding the name of XNNASDMN. There is no relevant information available about "XNNASDMN" based on the searches conducted. If you have specific context or additional details, please provide them, and I would be happy to assist you further.




🚀 Crew: crew
├── 📋 Task: 6a16152d-0ce4-4b1e-83d4-5f39fa526b77
│      Assigned to: Retriever Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Retriever Agent
│           Status: ✅ Completed
│       ├── 🔧 Used RAG Tool (7)
│       └── 🔧 Used Web Search Tool (5)
└── 📋 Task: cde4d9db-8cf0-49ec-a96c-a48cff42649f
       Status: Executing Task...
    └── 🤖 Agent: Senior Customer Support Agent
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 6a16152d-0ce4-4b1e-83d4-5f39fa526b77
│      Assigned to: Retriever Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Retriever Agent
│           Status: ✅ Completed
│       ├── 🔧 Used RAG Tool (7)
│       └── 🔧 Used Web Search Tool (5)
└── 📋 Task: cde4d9db-8cf0-49ec-a96c-a48cff42649f
       Assigned to: Senior Customer Support Agent
       Status: ✅ Completed
    └── 🤖 Agent: Senior Customer Support Agent
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: cde4d9db-8cf0-49ec-a96c-a48cff42649f                                                                     │
│  Agent: Senior Customer Support Agent                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 27c31765-1616-457b-82e7-a05a785dd68e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result_3.raw

'I apologize, but I do not have all the information I need to answer the query regarding the name of XNNASDMN. There is no relevant information available about "XNNASDMN" based on the searches conducted. If you have specific context or additional details, please provide them, and I would be happy to assist you further.'